In [ ]:
# библиотеки по работе с Телеграмом
import asyncio
import logging
from aiogram import Bot, Dispatcher, types
from aiogram.filters.command import Command
from aiogram import F
from aiogram.types import Message
from aiogram.types import FSInputFile
Token = "6398494394:AAHfuntYl5HTKh3EHtPZTv1OZlVRuPCKggs" # токен бота
logging.basicConfig(level = logging.INFO) # запуск логирования
bot = Bot(token = Token) # объект бота
dp = Dispatcher() # диспетчер

# библиотека для работы с yolo
from ultralytics import YOLO
model = YOLO('best.pt') # модель по лучшим параметрам, полученным при обучении в "YOLO_KaByte"

# библиотека для работы с http-запросами
import requests
# uri для получения пути к отправленной фотографии
URI_get_file_path = f'https://api.telegram.org/bot{Token}/getFile?file_id='
URI_get_file = f'https://api.telegram.org/file/bot{Token}/' # uri для получения файла по его пути

# библиотеки для работы с байтовыми изображениями
import io
from PIL import Image
from random import randint
import os

# хэндлер на команду /start
@dp.message(Command("start"))
async def cmd_start(message: types.Message):
    # о чём бот и для чего он создан?
    await message.answer("Привет, я ФотоБот! Отправь мне фотографию человека с камерой, а я \
попробую понять, на что он фотографирует. На данный момент я стараюсь предсказать 5 типов камер:\n\
1. цифровые;\n2. плёночные;\n3. мыльницы;\n4. моментальной печати;\n5. телефоны")

# хэндлер на получение фотографии
@dp.message(F.photo)
async def process_photo(message: Message):
    await message.answer("Обрабатываем фото...") # даём понять пользователю, что фото обрабатывается
    file_id = message.photo[-1].file_id # получаем id файла с самым высоким разрешением
    file_path = requests.get(URI_get_file_path + file_id).json()['result']['file_path'] # получаем путь к файлу
    photo_byte = requests.get(URI_get_file + file_path) # получаем фото в виде байтов
    photo = Image.open(io.BytesIO(photo_byte.content)) # получаем фотографию, с которой может работать yolo
    photo_predict = model(photo)[0] # делаем предсказание
    name = randint(10**20, 10**21) # уникальное имя фотографии
    photo_predict.save(f'predict/{name}.png') # сохраняем фотографию
    send = FSInputFile(f'predict/{name}.png') # отправляем сохранённое фото пользователю
    await message.answer_photo(photo = send) 
    os.remove(f'predict/{name}.png') # удалем фото, чтобы не занимать память

# запуск процесса поллинга новых апдейтов
async def main():
    await dp.start_polling(bot)
await main()